# PySpark - Fundamentos

Este notebook cobre os conceitos básicos de PySpark:
- SparkSession e configurações
- DataFrames e transformações
- Actions vs Transformations
- Window Functions
- Joins e Agregações

## 1. Criando SparkSession

In [ ]:
from pyspark.sql import SparkSession

# Forma básica
spark = SparkSession.builder \
    .appName("PySpark_Estudo") \
    .master("local[*]") \
    .getOrCreate()

print(f"Spark Version: {spark.version}")
print(f"App Name: {spark.sparkContext.appName}")

In [ ]:
# SparkSession com configurações avançadas
spark_configured = SparkSession.builder \
    .appName("SparkConfigurado") \
    .master("local[*]") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "2g") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .getOrCreate()

## 2. Criando DataFrames

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

# Dados de exemplo
data = [
    (1, "João", "Vendas", 5000.0, "2023-01-15"),
    (2, "Maria", "TI", 7500.0, "2022-06-20"),
    (3, "Pedro", "Vendas", 4500.0, "2023-03-10"),
    (4, "Ana", "RH", 6000.0, "2021-11-05"),
    (5, "Carlos", "TI", 8000.0, "2020-08-22"),
    (6, "Julia", "TI", 6500.0, "2023-07-01"),
]

columns = ["id", "nome", "departamento", "salario", "data_contratacao"]

# Criando DataFrame simples
df = spark.createDataFrame(data, columns)
df.show()

In [ ]:
# Com schema explícito (recomendado para produção)
schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("nome", StringType(), True),
    StructField("departamento", StringType(), True),
    StructField("salario", DoubleType(), True),
    StructField("data_contratacao", StringType(), True),
])

df_schema = spark.createDataFrame(data, schema)
df_schema.printSchema()

In [ ]:
# Lendo de arquivos (exemplos de sintaxe)
# CSV
# df_csv = spark.read.csv("path/to/file.csv", header=True, inferSchema=True)

# JSON
# df_json = spark.read.json("path/to/file.json")

# Parquet
# df_parquet = spark.read.parquet("path/to/file.parquet")

# Com opções
# df_csv = spark.read \
#     .option("header", "true") \
#     .option("inferSchema", "true") \
#     .option("delimiter", ";") \
#     .csv("path/to/file.csv")

## 3. Transformações Básicas

In [ ]:
from pyspark.sql.functions import col, lit, when, upper, current_date

# SELECT - selecionar colunas
df.select("nome", "salario").show()
df.select(col("nome"), col("salario") * 1.1).show()

In [ ]:
# FILTER / WHERE - filtrar linhas
df.filter(col("salario") > 5000).show()
df.where((col("departamento") == "TI") & (col("salario") >= 7000)).show()

In [ ]:
# WITHCOLUMN - adicionar/modificar colunas
df_transformed = df \
    .withColumn("salario_anual", col("salario") * 12) \
    .withColumn("bonus", col("salario") * 0.1) \
    .withColumn("nome_upper", upper(col("nome"))) \
    .withColumn("data_atual", current_date())

df_transformed.show()

In [ ]:
# WHEN/OTHERWISE - condicionais
df_categoria = df.withColumn(
    "categoria_salario",
    when(col("salario") < 5000, "Junior")
    .when(col("salario") < 7000, "Pleno")
    .otherwise("Senior")
)
df_categoria.show()

In [ ]:
# WITHCOLUMNRENAMED - renomear colunas
df.withColumnRenamed("nome", "funcionario").show()

# DROP - remover colunas
df.drop("data_contratacao").show()

## 4. Agregações

In [ ]:
from pyspark.sql.functions import sum, avg, count, min, max, countDistinct

# GroupBy com agregações
df.groupBy("departamento").agg(
    count("*").alias("total_funcionarios"),
    sum("salario").alias("total_salarios"),
    avg("salario").alias("media_salario"),
    min("salario").alias("menor_salario"),
    max("salario").alias("maior_salario")
).show()

In [ ]:
# Múltiplas agregações
df.groupBy("departamento").agg(
    countDistinct("nome").alias("funcionarios_unicos"),
    (sum("salario") / count("*")).alias("media_manual")
).show()

## 5. Window Functions

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, rank, dense_rank, lead, lag

# Definindo window
window_dept = Window.partitionBy("departamento").orderBy(col("salario").desc())

df_window = df \
    .withColumn("rank_salario", rank().over(window_dept)) \
    .withColumn("dense_rank_salario", dense_rank().over(window_dept)) \
    .withColumn("row_num", row_number().over(window_dept))

df_window.show()

In [ ]:
# Lead e Lag - próximo e anterior
window_ordered = Window.orderBy("salario")

df.withColumn("proximo_salario", lead("salario", 1).over(window_ordered)) \
    .withColumn("salario_anterior", lag("salario", 1).over(window_ordered)) \
    .show()

In [ ]:
# Running total (soma acumulada)
window_running = Window.partitionBy("departamento") \
    .orderBy("id") \
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)

df.withColumn("salario_acumulado", sum("salario").over(window_running)).show()

## 6. Joins

In [ ]:
from pyspark.sql.functions import broadcast

# Criando tabela de departamentos
departamentos = spark.createDataFrame([
    ("Vendas", "São Paulo"),
    ("TI", "Rio de Janeiro"),
    ("RH", "Belo Horizonte"),
], ["departamento", "cidade"])

departamentos.show()

In [ ]:
# Inner Join
df.join(departamentos, "departamento", "inner").show()

In [ ]:
# Left Join
df.join(departamentos, "departamento", "left").show()

In [ ]:
# Broadcast Join (otimização para tabelas pequenas)
df.join(broadcast(departamentos), "departamento").show()

## 7. Actions (executam o DAG)

In [ ]:
# Actions comuns
print("Count:", df.count())
print("First:", df.first())
print("Take 3:", df.take(3))

In [ ]:
# Estatísticas
df.describe("salario").show()
df.summary().show()

## 8. Repartition e Coalesce

In [ ]:
print("Partições originais:", df.rdd.getNumPartitions())

# Repartition - aumenta ou diminui (shuffle completo)
df_repartitioned = df.repartition(4)
print("Após repartition(4):", df_repartitioned.rdd.getNumPartitions())

# Coalesce - apenas diminui (sem shuffle)
df_coalesced = df_repartitioned.coalesce(2)
print("Após coalesce(2):", df_coalesced.rdd.getNumPartitions())

In [ ]:
# Repartition por coluna - dados da mesma chave na mesma partição
df_by_dept = df.repartition("departamento")
print("Repartition por departamento:", df_by_dept.rdd.getNumPartitions())

## 9. Cache e Persist

In [ ]:
from pyspark.storagelevel import StorageLevel

# Cache - armazena em memória
df.cache()  # Equivalente a persist(StorageLevel.MEMORY_ONLY)

# Persist com opções
# StorageLevel.MEMORY_ONLY - só memória
# StorageLevel.MEMORY_AND_DISK - memória com spillover para disco
# StorageLevel.DISK_ONLY - só disco
df.persist(StorageLevel.MEMORY_AND_DISK)

# Remover do cache
df.unpersist()

## 10. Explain - Plano de Execução

In [ ]:
# Ver plano de execução
df.filter(col("salario") > 5000) \
    .groupBy("departamento") \
    .count() \
    .explain(True)

In [ ]:
# Finalizando a sessão
# spark.stop()